In [1]:
import requests
import time
import pandas as pd

API_KEY = {'X-API-Key': '69LC227X'}
s = requests.Session()
s.headers.update(API_KEY)

def get_position(session, ticker):
    resp = session.get('http://localhost:9999/v1/securities')
    securitieslist = resp.json()
    for x in securitieslist:
        if x['ticker'] == ticker:
            position = x['position']
    return position

def get_orderbook(session, ticker):
    # Return DataFrame of cleaned order book
    info = {"ticker": ticker, "limit": 100000}
    resp = session.get('http://localhost:9999/v1/securities/book', params=info)
    book = resp.json()

    bid_df = pd.DataFrame(book['bids'])
    ask_df = pd.DataFrame(book['asks'])

    return bid_df, ask_df

def trade_mispricing(ordersize=10, max_position=100, lower_threshold=-1.5, upper_threshold=1.5, target=1, tolerance=0.1):
    position_open = False

    while True:
        # Get the latest bid and ask prices for CL-1F and CL-2F
        cl1f_bid, cl1f_ask = get_orderbook(s, 'CL-1F')
        cl2f_bid, cl2f_ask = get_orderbook(s, 'CL-2F')

        # If either order book is empty, skip this iteration
        if cl1f_bid.empty or cl2f_bid.empty:
            time.sleep(5)
            continue

        # Calculate the price difference using the best bid prices
        price_difference = cl2f_bid.iloc[0]['price'] - cl1f_bid.iloc[0]['price']

        # Get positions of CL-1F and CL-2F
        cl1f_position = get_position(s, 'CL-1F')
        cl2f_position = get_position(s, 'CL-2F')

        # If the price difference is close enough to the target value and a position is open, close the position
        if abs(price_difference - target) <= tolerance and position_open:
            if cl1f_position > 0:
                s.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': cl1f_position, 'action': 'SELL'})
            elif cl1f_position < 0:
                s.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': -cl1f_position, 'action': 'BUY'})

            if cl2f_position > 0:
                s.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': cl2f_position, 'action': 'SELL'})
            elif cl2f_position < 0:
                s.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': -cl2f_position, 'action': 'BUY'})

            position_open = False

        # If the price difference is outside the tolerance range and no position is open, open a new position
        elif not position_open and (price_difference > upper_threshold or price_difference < lower_threshold):
            if price_difference > upper_threshold and abs(cl2f_position - cl1f_position) < max_position:
                # Short the overpriced CL-2F and go long on the underpriced CL-1F
                s.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': ordersize, 'action': 'SELL'})
                s.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': ordersize, 'action': 'BUY'})
                position_open = True

            elif price_difference < lower_threshold and abs(cl1f_position - cl2f_position) < max_position:
                # Short the overpriced CL-1F and go long on the underpriced CL-2F
                s.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': ordersize, 'action': 'SELL'})
                s.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': ordersize, 'action': 'BUY'})
                position_open = True

        # Wait for a certain time period before checking the prices again
        time.sleep(1)


trade_mispricing()


KeyError: 'bids'